In [1]:
from google.cloud import storage
import os
from datetime import date, timedelta
import datetime as dt
import pandas as pd
from alpaca_trade_api.rest import REST, TimeFrame

API_KEY = 'PKWZQ9TTQYG9H00RUGS0'
SECRET_KEY = '7C45wJDOuh5drGCarAa9JIKviKSeicitFYIZTpXQ'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'servicekey_googlecloud.json'

In [2]:
def get_date_market_data(API_KEY, SECRET_KEY, market="SPY", timeframe=TimeFrame.Minute, extended_hours=False, exchange="IEX", write=True, date_run="2022-01-04"):

    #dt_string = dt.datetime.strptime(f"{date_run}T00:00:00+00:00", "%Y-%m-%dT%H:%M:%S%z")
    dt_string = date_run
    #path="./snp_data.csv"
    path = "./%s.snp.csv"%date_run
    #yesterday = dt.datetime.now(dt.timezone.utc) - dt.timedelta(days=1)

    date_market_open = date_run
    date_market_close = date_run

    alpaca_rest_client = REST(API_KEY, SECRET_KEY)

    data = alpaca_rest_client.get_bars(market, TimeFrame.Minute, date_market_open, date_market_close).df    

    data.to_csv(path)
    upload_to_bucket()

    return print("Downloading data for: "+ date_run)


In [3]:
get_date_market_data(API_KEY, SECRET_KEY, write=True, date_run="2022-01-06")

NameError: name 'upload_to_bucket' is not defined

In [93]:
def backfill_date(date_run = '2022-02-15'):
    #date_run = '2022-02-15'
    start_date = dt.datetime.strptime(date_run, "%Y-%m-%d")
    end_date = dt.datetime.today() - timedelta(days = 1)
    day_count = (end_date - start_date).days + 1
    
    for single_date in (start_date + timedelta(n) for n in range(day_count)):
        single_date = dt.datetime.strftime(single_date, "%Y-%m-%d")
        get_date_market_data(API_KEY, SECRET_KEY, write=True, date_run=single_date)
    
    return print("Backfill for SPY complete from " + date_run + " to " + dt.datetime.strftime(end_date, "%Y-%m-%d"))


In [94]:
backfill_date(date_run = '2022-02-15')

Backfill for SPY complete from 2022-02-15 to 2022-02-16


In [104]:
def upload_to_bucket(bucket_name = 'alpaca-data', exchange = "IEX", symbol = "SPY", date_run = "2022-01-04"):
    bucket = storage.Client().bucket(bucket_name)
#     yesterday = dt.datetime.now(dt.timezone.utc) - dt.timedelta(days=1)

#     date_run = yesterday

    base_dst = "datatype=exchange={exchange}/symbol={symbol}/year={year}/month={month}/day={day}/{name}.csv"
    
    date_run = dt.datetime.strptime(f"{date_run}", "%Y-%m-%d")
    
    dst = base_dst.format(
    date = date_run,
    exchange = exchange,
    symbol = symbol,
    year= date_run.year,
    month=date_run.month,
    day=date_run.day,
    name=f"{symbol}_{dt.datetime.strftime(date_run,'%Y-%m-%d')}")
    
    local_path = './snp_data.csv'
    blob = bucket.blob(dst)
    blob.upload_from_filename(local_path)
    print(bucket)
    os.remove("snp_data.csv")
    print("File removed locally")

In [105]:
upload_to_bucket(date_run = "2022-01-01")

<Bucket: alpaca-data>
File removed locally
